In [ ]:
import math
from functools import partial

import numpy as np
import pandas as pd
import pygame
from IPython.core.display import Image

from island_influence.utils import deterministic_ring, random_ring

%load_ext autoreload

In [ ]:
seed = None

num_harvesters = 4
num_excavators = 4
num_obstacles = 4
num_pois = 4

scale_factor = 1

harvester_bounds = np.asarray([0, 3]) * scale_factor
excavator_bounds = np.asarray([0, 3]) * scale_factor
obstacle_bounds = np.asarray([5, 8]) * scale_factor
poi_bounds = np.asarray([10, 13]) * scale_factor
center_loc = (5, 5)

types_num = {
    'harvester': num_harvesters,
    'excavator': num_excavators,
    'obstacle': num_obstacles,
    'poi': num_pois,
}

_info = {
    'harvester': {'observation_radius': 5, 'size': 1, 'weight': 1, 'value': 1, 'agent_type': 0},
    'excavator': {'observation_radius': 5, 'size': 1, 'weight': 1, 'value': 1, 'agent_type': 1},
    'obstacle': {'observation_radius': 1, 'size': 1, 'weight': 1, 'value': 1, 'agent_type': 2},
    'poi': {'observation_radius': 1, 'size': 1, 'weight': 1, 'value': 1, 'agent_type': 3},
}

type_map = {
    info['agent_type']: agent_type
    for agent_type, info in _info.items()
}

harvester_locs = partial(random_ring, center=center_loc, min_rad=harvester_bounds[0], max_rad=harvester_bounds[1])
excavator_locs = partial(random_ring, center=center_loc, min_rad=excavator_bounds[0], max_rad=excavator_bounds[1])
obstacle_locs = partial(deterministic_ring, center=center_loc, radius=np.average(obstacle_bounds))
poi_locs = partial(deterministic_ring, center=center_loc, radius=np.average(poi_bounds))
location_funcs = {
    'harvesters': harvester_locs,
    'excavators': excavator_locs,
    'obstacles': obstacle_locs,
    'pois': poi_locs,
}

In [ ]:
# create set of points for harvesters
# create set of points for excavators
# create set of points for obstacles
# create set of points for pois
harvester_locs = location_funcs['harvesters'](num_points=types_num['harvester'], seed=seed)
excavator_locs = location_funcs['excavators'](num_points=types_num['excavator'], seed=seed)
obstacle_locs = location_funcs['obstacles'](num_points=types_num['obstacle'], seed=seed)
poi_locs = location_funcs['pois'](num_points=types_num['poi'], seed=seed)
locs = {'harvester': harvester_locs, 'excavator': excavator_locs, 'obstacle': obstacle_locs, 'poi': poi_locs}

keys = ['harvester', 'excavator', 'obstacle', 'poi']
entries = [
    {'name': f'{each_key}:{loc_idx}', **{f'location_{idx}': val for idx, val in enumerate(each_loc)}, **_info[each_key]}
    for each_key in keys
    for loc_idx, each_loc in enumerate(locs[each_key])
]
state = pd.DataFrame(entries)

In [ ]:
render_scale = 2
agent_locations = state.loc[:, state.columns.str.startswith('loc')]

min_loc = np.min(agent_locations, axis=0)
max_loc = np.max(agent_locations, axis=0)

delta_x = max_loc[0] - min_loc[0]
delta_y = max_loc[1] - min_loc[1]

render_bound = math.ceil(max(delta_x, delta_y))
render_bound = render_bound * render_scale
location_offset = render_bound // (render_scale * 2)

In [ ]:
window_size = 512

black = (0, 0, 0)
white = (255, 255, 255)

# The size of a single grid square in pixels
pix_square_size = (window_size / render_bound)

agent_colors = {'harvester': (0, 255, 0), 'excavator': (0, 0, 255), 'obstacle': black, 'poi': (255, 0, 0)}
default_color = (128, 128, 128)

agent_sizes = {'harvester': 0.25, 'excavator': 0.25, 'obstacle': 0.25, 'poi': 0.25}
default_size = 0.1
size_scalar = 1
size_width = 2
obs_width = 1

text_size = 14
write_values = False
write_legend = True
render_fps = 4

In [ ]:
def get_object_state(full_state: pd.DataFrame, object_name):
    object_state = state[full_state['name'] == object_name]
    object_state = object_state.iloc[0]
    return object_state

def get_object_location(object_state: pd.Series):
    a_locations = object_state.loc[['location_0', 'location_1']]
    return a_locations

In [ ]:
pygame.init()
pygame.display.init()
pygame.font.init()
window = pygame.display.set_mode((window_size, window_size))
clock = pygame.time.Clock()

canvas = pygame.Surface((window_size, window_size))
canvas.fill(white)
pygame.font.init()
font = pygame.font.SysFont('arial', text_size)

for ridx, agent in state.iterrows():  
    agent_idx = agent['agent_type']
    obs_radius = agent['observation_radius']
    size_radius = agent['size']
    agent_value = agent['value']

    # # only render the object if it has not been removed by an excavator
    # if agent_value <= 0:
    #     continue
    
    agent_type = type_map[agent_idx]
    location = get_object_location(agent).to_numpy()
    location = location + location_offset
    
    acolor = agent_colors.get(agent_type, default_color)
    asize = agent_sizes.get(agent_type, default_size)
    asize *= size_scalar
    
    pygame.draw.circle(canvas, acolor, (location + 0.5) * pix_square_size, pix_square_size * size_radius, width=size_width)
    pygame.draw.circle(canvas, black, (location + 0.5) * pix_square_size, pix_square_size * asize)
    # pygame.draw.circle(canvas, acolor, (location + 0.5) * pix_square_size, pix_square_size * obs_radius, width=obs_width)

# The following line copies our drawings from `canvas` to the visible window
window.blit(canvas, canvas.get_rect())
pygame.event.pump()
pygame.display.update()

# We need to ensure that human-rendering occurs at the predefined framerate.
# The following line will automatically add a delay to keep the framerate stable.
clock.tick(render_fps)

pygame.display.flip()
pygame.image.save(window, 'frame.png')
display(Image(filename='frame.png'))

pygame.quit()